In [48]:
import numpy as np
import cv2

image = cv2.imread('test_image.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [49]:
from narya.models.keras_models import DeepHomoModel

direct_homography_model = DeepHomoModel()

C:\Users\ADMIN\anaconda3\envs\narya\lib\site-packages\keras\layers\core\lambda_layer.py:305: UserWarning: onnx2keras.pooling_layers is not loaded, but a Lambda layer uses it. It may cause errors.
  'function_type')
C:\Users\ADMIN\anaconda3\envs\narya\lib\site-packages\keras\layers\core\lambda_layer.py:305: UserWarning: onnx2keras.reshape_layers is not loaded, but a Lambda layer uses it. It may cause errors.
  'function_type')


In [50]:
import tensorflow as tf

In [51]:
WEIGHTS_PATH = (
    "https://storage.googleapis.com/narya-bucket-1/models/deep_homo_model.h5"
)
WEIGHTS_NAME = "deep_homo_model.h5"
WEIGHTS_TOTAR = False

checkpoints = tf.keras.utils.get_file(
                WEIGHTS_NAME, WEIGHTS_PATH, WEIGHTS_TOTAR,
            )

direct_homography_model.load_weights(checkpoints)

Succesfully loaded weights from C:\Users\ADMIN\.keras\datasets\deep_homo_model.h5


In [55]:
from narya.models.keras_models import KeypointDetectorModel
from narya.utils.homography import *

In [53]:
import kornia as K

In [61]:
def get_perspective_transform_torch(src, dst):
    """Get the homography matrix between src and dst

    Arguments:
        src: Tensor of shape (B,4,2), the four original points per image
        dst: Tensor of shape (B,4,2), the four corresponding points per image
    Returns:
        A tensor of shape (B,3,3), each homography per image
    Raises:

    """
    return K.geometry.get_perspective_transform(src, dst)

In [59]:
def compute_homography(batch_corners_pred):
    """Compute the homography from the predictions of DeepHomoModel

    Arguments:
        batch_corners_pred: np.array of shape (B,8) with the predictions
    Returns:
        np.array of shape (B,3,3) with the homographies
    Raises:
        
    """
    batch_size = batch_corners_pred.shape[0]
    corners = get_corners_from_nn(batch_corners_pred)
    orig_corners = get_default_corners(batch_size)
    homography = get_perspective_transform_torch(
        to_torch(orig_corners), to_torch(corners)
    )
    return to_numpy(homography)

In [62]:
corners = direct_homography_model(image)
template = cv2.imread('world_cup_template.png')
template = cv2.cvtColor(template, cv2.COLOR_BGR2RGB)
template = cv2.resize(template, (1280,720))/255.
pred_homo = compute_homography(corners)[0]
print("Predicted homography: {}".format(pred_homo))

Predicted homography: [[ 3.3973148e-01 -1.2924273e-03  4.6077055e-01]
 [ 3.0281150e-01  1.7163653e+00  4.9089946e-02]
 [ 4.1546315e-02  1.5496986e+00  1.0000000e+00]]
